# Coronavirus Latest
> Updates on the respiratory illness that has infected more than one million people and killed tens of thousands.

- toc:false
- branch: master
- badges: false
- hide_colab_badge: true
- comments: false
- permalink:/covid-overview/


In [0]:
#hide
#@title Import modules and load datasets from source
import pandas as pd
import numpy as np
import altair as alt
from IPython.display import HTML
import plotly.graph_objects as go
from plotly.subplots import make_subplots

base_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data'

# confirmed, deaths, recovered
def get_time_series(type):
  _url = f'{base_url}/csse_covid_19_time_series/time_series_covid19_{type}_global.csv'
  _dff = pd.read_csv(_url)
  _cols = _dff.columns[~_dff.columns.isin(['Province/State', 'Country/Region', 'Lat', 'Long'])]
  _dff = (_dff.groupby('Country/Region')[_cols].sum().stack().reset_index(name = 'Cases')
        .rename(columns = {'level_1': 'Date', 'Country/Region': 'Country'}))
  _dff['Date'] = pd.to_datetime(_dff['Date'], format='%m/%d/%y')
  return _dff

df_confirmed = get_time_series('confirmed')
df_recovered = get_time_series('recovered')
df_deaths = get_time_series('deaths')
date_latest = df_confirmed['Date'].max()

df_last = lambda name, _df, n: _df[_df['Date'].isin(_df['Date'].tail(n))].set_index(['Country', 'Date']).rename(columns={'Cases': name})
df_summary = lambda last_n: pd.concat([df_last('Confirmed', df_confirmed, last_n), df_last('Deaths', df_deaths, last_n), df_last('Recovered', df_recovered, last_n)], axis=1)
df_global = df_summary(2).groupby(['Date']).sum().reset_index()
s_date_latest = date_latest.strftime("%B %d, %Y")
n_confirmed = df_global.loc[1]['Confirmed']
n_deaths = df_global.loc[1]['Deaths']
n_recovered = df_global.loc[1]['Recovered']

df_global_diff = df_global.diff()
n_confirmed_diff = df_global_diff.loc[1]['Confirmed']
n_deaths_diff = df_global_diff.loc[1]['Deaths']
n_recovered_diff = df_global_diff.loc[1]['Recovered']
countries = df_confirmed['Country'].unique()

## Overview

In [2]:
#hide_input
fig = go.Figure()

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = n_confirmed,
    # number = {'valueformat': ','},
    align = "left",
    title = {"text": f'Total cases reported worldwide in {len(countries) - 1}, {s_date_latest}', "align": "left", "font": {"family": "sans-serif"}},
    delta = {'reference': n_confirmed - n_confirmed_diff, 'position': 'right', 'valueformat': ','},
    domain = {'x': [0, .6], 'y': [.9, 1]}))

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = n_recovered,
    # number = {'valueformat': ','},
    # number = {'prefix': f'{n_recovered/n_confirmed*100:.2f}% or '},
    align = "left",
    title = {"text": f'total of {n_recovered/n_confirmed*100:.2f}% has recovered', "align": "left", "font": {"family": "sans-serif"}},
    delta = {'reference': n_recovered - n_recovered_diff, 'position': 'right', 'valueformat': ','},
    domain = {'x': [0, .6], 'y': [.6, .7]}))

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = n_deaths,
    # number = {'valueformat': ','},
    align = "left",
    title = {"text": f'total of {n_deaths/n_confirmed*100:.2f}% deaths reported', "align": "left", "font": {"family": "sans-serif"}},
    delta = {'reference': n_deaths - n_deaths_diff, 'position': 'right', 'valueformat': ','},
    domain = {'x': [0, .6], 'y': [.3, .4]}))

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = n_confirmed - n_deaths - n_recovered,
    # number = {'valueformat': ','},
    align = "left",
    title = {"text": f'total of {(n_confirmed - n_deaths - n_recovered)/n_confirmed*100:.2f}% remain active', "align": "left", "font": {"family": "sans-serif"}},
    delta = {'reference': (n_confirmed -  n_confirmed_diff) - (n_deaths - n_deaths_diff) - (n_recovered - n_recovered_diff), 'position': 'right', 'valueformat': ','},
    domain = {'x': [0, .6], 'y': [0, .1]}))


fig.show()

### Confirmed Cases

In [3]:
#hide_input
_df = df_confirmed.groupby(['Date']).sum().unstack().reset_index().drop(columns=['level_0']).rename(columns={0: 'Cummulative Confirmed'})
_df['New Confirm'] = _df[['Cummulative Confirmed']].diff()

_df_line = alt.Chart(_df).mark_line(color='#D32F2F').encode(
    alt.X('Date:T'),
    alt.Y('Cummulative Confirmed:Q', axis=alt.Axis(titleColor='#D32F2F'), scale=alt.Scale(type='log')),
    tooltip=list(_df)
)

_df_new_bar = alt.Chart(_df).mark_bar(opacity=0.3, color='#303F9F').encode(
    alt.X('Date:T'),
    alt.Y('New Confirm:Q', axis=alt.Axis(titleColor='#303F9F'), scale=alt.Scale(type='log')),
    tooltip=list(_df)
)


alt.layer(_df_line, _df_new_bar).resolve_scale(
    y='independent'
).properties(
    title = 'Confirmed Cases across the world',
    width = 600
)

alt.LayerChart(...)

In [0]:
#hide
def df_summary(_df):
  _summary = _df.groupby(['Date', 'Country']).sum().unstack().xs('Cases', axis=1).tail(2).reset_index(drop=True).transpose()
  _summary['New'] = _summary.diff(axis=1)[1]
  _df_new_mean =_df.groupby(['Date', 'Country']).sum().unstack().xs('Cases', axis=1).diff().rolling(window=7).mean()[-1:] \
                        .reset_index(drop=True).transpose().round(0).sort_values(by=0,ascending=False).rename(columns={0: 'New_Mean_7D'})                        
  _df_pct_chg = _df.groupby(['Date', 'Country']).sum().unstack().xs('Cases', axis=1).pct_change().rolling(window=7).mean()[-1:] \
                        .reset_index(drop=True).transpose().sort_values(by=0,ascending=False).rename(columns={0: 'Change_Mean_7D'})
  #df_confirmed_pct_mean_latest = df_confirmed_pct_mean
  _summary = pd.merge(_summary, _df_new_mean, on = 'Country')

  _summary['Change'] = _summary.pct_change(axis=1)[1]
  _summary = pd.merge(_summary, _df_pct_chg, on = 'Country')
  _summary['Trending'] = _summary['Change'] - _summary['Change_Mean_7D']
  _summary['Weight'] = _summary['New'] / _summary['New'].sum()
  _summary = _summary.drop(columns=0).rename(columns={1: 'Total'}).sort_values(by=['New', 'Total', 'Trending'], ascending=False).round(4)
  return _summary

df_summary_confirmed = df_summary(df_confirmed)

In [5]:
#hide_input
_data = df_summary_confirmed.reset_index().head(50)
_new_bar = alt.Chart().mark_bar().encode(
    alt.X('Country:N', sort='-y'),
    alt.Y('New:Q', scale=alt.Scale(type='log')),
    tooltip = list(df_summary_confirmed)
)

_mean_bar = alt.Chart().mark_tick(color='red').encode(
    alt.X('Country:N'),# axis=alt.Axis(labels=False, ticks=False)),
    alt.Y('New_Mean_7D:Q', scale=alt.Scale(type='log')),
    tooltip = list(df_summary_confirmed)
)


_new_text = _new_bar.mark_text(
    angle=270,
    align='left',
    baseline='middle',    
    dy=3,
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='New:Q'
)

alt.layer(_new_bar + _new_text, data=_data).properties(
  title = f'New Confirmed Cases On {s_date_latest}',
  width= 600
)#.resolve_scale(x='independent')

alt.LayerChart(...)

In [6]:
#hide
df_summary_confirmed.sort_values(by='New', ascending=False).head(50)

,Total,New,New_Mean_7D,Change,Change_Mean_7D,Trending,Weight
Country,,,,,,,
US,1180375,22335.0,27454.0,0.0193,0.0257,-0.0064,0.2926
Russia,145268,10581.0,8303.0,0.0786,0.0758,0.0028,0.1386
Brazil,108620,6794.0,5882.0,0.0667,0.0706,-0.0039,0.0890
United Kingdom,191832,3990.0,4783.0,0.0212,0.0278,-0.0066,0.0523
India,46437,3932.0,2427.0,0.0925,0.0673,0.0252,0.0515
Ecuador,31881,2343.0,1234.0,0.0793,0.0465,0.0328,0.0307
Saudi Arabia,28656,1645.0,1406.0,0.0609,0.0620,-0.0011,0.0216
Turkey,127659,1614.0,2200.0,0.0128,0.0185,-0.0057,0.0211
Canada,61957,1453.0,1763.0,0.0240,0.0323,-0.0082,0.0190


In [7]:
#hide_input
_data = df_summary_confirmed[df_summary_confirmed['Total'] > 1000].reset_index()

_new_bar = alt.Chart().mark_bar().encode(
    alt.X('Country:N', sort='-y'),
    alt.Y('Change:Q', axis=alt.Axis(format='%')),
    tooltip = list(df_summary_confirmed)
)

_mean_bar = alt.Chart().mark_tick(color='red').encode(
    alt.X('Country:N'),# axis=alt.Axis(labels=False, ticks=False)),
    alt.Y('Change_Mean_7D:Q',axis=alt.Axis(format='%')),
    tooltip = list(df_summary_confirmed)
)

_new_text = _new_bar.mark_text(
    size=10,
    angle=270,
    align='left',
    baseline='middle', 
    dy=3,
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='Change:Q'
)

alt.layer(_new_bar + _new_text, data=_data).properties(
  title = f'Change of New Confirmed Cases (and 7 Days Average) On {s_date_latest}',
  width= 600
)#.resolve_scale(x='independent')

#alt.layer(_mean_bar, data=_data)

alt.LayerChart(...)

## Weight of Cases by Countries

In [8]:
#hide_input
df_confirmed_new = df_confirmed.set_index(['Date', 'Country']).unstack().diff().stack().reset_index()
df_confirmed_new_pct = df_confirmed_new.set_index(['Date', 'Country']).rename(columns={'Cases': 'New Confirm Weight'}).unstack().apply(lambda x: x/sum(x), axis=1).round(3).stack().reset_index()
selection = alt.selection_multi(fields=['Country'], bind='legend')
df_confirmed_new_pct = df_confirmed_new_pct[df_confirmed_new_pct['New Confirm Weight'] > 0.03]

_weight_bar = alt.Chart(df_confirmed_new_pct).mark_bar().encode(
    x='Date:T', 
    y=alt.Y('New Confirm Weight:Q', axis=alt.Axis(format='%'), title='Daily Confirm Weight'), 
    color='Country',
    tooltip=list(df_confirmed_new_pct),
    opacity=alt.condition(selection,alt.value(1), alt.value(0.2))
).add_selection(selection)
_weight_bar.properties(
    title = 'Weight of New Cases by Country (Country with weight more than 3%)',
    width = 600
)

alt.Chart(...)

In [9]:
#hide_input
df_summary_deaths = df_summary(df_deaths)
df_deaths_major = df_summary_deaths[(df_summary_deaths['New'] > 10) & (df_summary_deaths['Total'] > 100)].reset_index()

df_deaths_major.sort_values(by='New', ascending=False)


,Country,Total,New,New_Mean_7D,Change,Change_Mean_7D,Trending,Weight
0,US,68922,1240.0,1809.0,0.0183,0.0295,-0.0111,0.3049
1,Brazil,7367,316.0,395.0,0.0448,0.0697,-0.0249,0.0777
2,France,25204,304.0,273.0,0.0122,0.0113,0.0009,0.0747
3,United Kingdom,28809,289.0,622.0,0.0101,0.0237,-0.0136,0.0711
4,Canada,4003,208.0,166.0,0.0548,0.0503,0.0045,0.0511
5,Italy,29079,195.0,300.0,0.0068,0.0108,-0.0040,0.0479
6,India,1566,175.0,90.0,0.1258,0.0760,0.0498,0.0430
7,Spain,25428,164.0,272.0,0.0065,0.0112,-0.0047,0.0403
8,Germany,6993,127.0,124.0,0.0185,0.0191,-0.0006,0.0312
9,Mexico,2271,117.0,120.0,0.0543,0.0681,-0.0138,0.0288


In [10]:
#hide_input
alt.Chart(df_deaths_major).mark_bar().encode(
    alt.X('Country:N', sort='-y'),
    alt.Y('Change_Mean_7D:Q', axis=alt.Axis(format='%'), title='Mean Change'),
    tooltip=list(df_deaths_major)
).properties(
    width=600,
    title='Percentage Change (Mean of 7 Days) of Deaths Case'
)

alt.Chart(...)

In [11]:
#hide
df_confirmed_deaths = pd.merge(df_summary_confirmed.reset_index()[['Country', 'Total']].rename(columns={'Total': 'Confirmed'}), \
df_summary_deaths.reset_index()[['Country', 'Total']].rename(columns={'Total': 'Deaths'}), 
on='Country')

df_confirmed_deaths['Deaths Rate'] = df_confirmed_deaths['Deaths'] / df_confirmed_deaths['Confirmed']
df_confirmed_deaths[df_confirmed_deaths['Confirmed'] > 1000].sort_values(by='Deaths Rate', ascending=False).reset_index(drop=True).head(50)


,Country,Confirmed,Deaths,Deaths Rate
0,Belgium,50267,7924,0.157638
1,United Kingdom,191832,28809,0.150178
2,France,169583,25204,0.148623
3,Italy,211938,29079,0.137205
4,Netherlands,40968,5098,0.124439
5,Sweden,22721,2769,0.121870
6,Spain,218011,25428,0.116636
7,Hungary,3035,351,0.115651
8,Algeria,4648,465,0.100043
9,Mexico,24905,2271,0.091187


## Cases in US

In [12]:
#hide 
def get_country(code):
  _df = [
          df_confirmed[df_confirmed['Country'] == code][['Date', 'Cases']].set_index('Date'), 
          df_deaths[df_deaths['Country'] == code][['Date', 'Cases']].set_index('Date').rename(columns={'Cases': 'Deaths'}),
          df_recovered[df_recovered['Country'] == code][['Date', 'Cases']].set_index('Date').rename(columns={'Cases': 'Recovered'})
        ]
  _df = pd.concat(_df, axis=1, join='inner')
  _df['Active'] = _df['Cases'] - _df['Deaths'] - _df['Recovered']
  _df = pd.concat( [_df, _df.diff().rename(columns={'Cases': 'New Cases', 'Deaths': 'New Deaths', 'Recovered': 'New Recovered', 'Active': 'New Active'})], axis=1, join='inner')
  _df['Deaths_Rate'] = _df['Deaths'] / _df['Cases']
  _df['Recovered_Rate'] = _df['Recovered'] / _df['Cases']
  return _df.reset_index()

country = 'US'
df_country_cases = get_country(country)
df_country_cases

,Date,Cases,Deaths,Recovered,Active,New Cases,New Deaths,New Recovered,New Active,Deaths_Rate,Recovered_Rate
0,2020-01-22,1,0,0,1,NaN,NaN,NaN,NaN,0.000000,0.000000
1,2020-01-23,1,0,0,1,0.0,0.0,0.0,0.0,0.000000,0.000000
2,2020-01-24,2,0,0,2,1.0,0.0,0.0,1.0,0.000000,0.000000
3,2020-01-25,2,0,0,2,0.0,0.0,0.0,0.0,0.000000,0.000000
4,2020-01-26,5,0,0,5,3.0,0.0,0.0,3.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
99,2020-04-30,1069424,62996,153947,852481,29515.0,2029.0,33227.0,-5741.0,0.058906,0.143953
100,2020-05-01,1103461,64943,164015,874503,34037.0,1947.0,10068.0,22022.0,0.058854,0.148637
101,2020-05-02,1132539,66369,175382,890788,29078.0,1426.0,11367.0,16285.0,0.058602,0.154857
102,2020-05-03,1158040,67682,180152,910206,25501.0,1313.0,4770.0,19418.0,0.058445,0.155566


In [13]:
#hide_input
_new_bar = alt.Chart(df_country_cases).mark_bar().encode(
    alt.X('Date:T'),
    alt.Y('New Cases:Q'),
    tooltip = list(df_country_cases)
)

_total_line = alt.Chart(df_country_cases[df_country_cases['Cases'] > 10]).mark_line(color='#57A44C').encode(
    alt.X('Date:T'),
    alt.Y('Cases:Q', axis=alt.Axis(titleColor='#57A44C'), scale=alt.Scale(type='log')),
    tooltip = list(df_country_cases)
)

alt.layer(_total_line, _new_bar).properties(
    width = 600,
    title = f'Number of Total Cases (Daily New Cases) for {country}'
).resolve_scale(y='independent')

alt.LayerChart(...)

In [14]:
#hide_input
_new_bar = alt.Chart(df_country_cases).mark_bar().encode(
    alt.X('Date:T'),
    alt.Y('New Deaths:Q'),
    tooltip = list(df_country_cases)
)

_total_line = alt.Chart(df_country_cases[df_country_cases['Deaths'] > 10]).mark_line(color='#57A44C').encode(
    alt.X('Date:T'),
    alt.Y('Deaths:Q', axis=alt.Axis(titleColor='#57A44C'), scale=alt.Scale(type='log')),
    tooltip = list(df_country_cases)
)

alt.layer(_total_line, _new_bar).properties(
    width = 600,
    title = f'Number of Total Deaths Cases (Daily Deaths Cases) for {country}'
).resolve_scale(y='independent')

alt.LayerChart(...)

In [15]:
#hide_input
_data = df_country_cases[(df_country_cases['New Cases']) > 0 & (df_country_cases['Cases'] > 100)]
alt.Chart(_data).mark_line().encode(
    alt.X('Cases:Q', scale=alt.Scale(type='log')),
    alt.Y('New Cases:Q', scale=alt.Scale(type='log', domain=[0.1, 100000]))
).properties(width = 600)

alt.Chart(...)

In [16]:
#hide_input
_rate1_line = alt.Chart(df_country_cases).mark_line(color='red').encode(
    alt.X('Date:T'),
    alt.Y('Deaths_Rate:Q', axis=alt.Axis(format='%', titleColor='red')),
    tooltip = list(df_country_cases)
) 

_rate2_line = alt.Chart(df_country_cases).mark_line(color='green').encode(
    alt.X('Date:T'),
    alt.Y('Recovered_Rate:Q', axis=alt.Axis(format='%', titleColor='green')),
    tooltip = list(df_country_cases)
)

(_rate1_line + _rate2_line).properties(
    width = 600,
    title = f'Rate of Deaths and Recovered for {country}'
).resolve_scale(y='independent')

alt.LayerChart(...)

## Deaths Cases Global


In [17]:
#hide_input
countries = ['China', 'Korea, South', 'Italy', 'Spain', 'US', 'Belgium']
_df = df_deaths.pivot(index='Country', columns='Date').diff(axis=1)['Cases'].stack().reset_index().rename(columns={0: 'New Deaths'})
_data = _df[_df['Country'].isin(countries)]

selection = alt.selection_multi(fields=['Country'], bind='legend')
alt.Chart(_data).mark_bar(opacity = 0.8).encode(
    alt.X('Date:T'),
    alt.Y('New Deaths:Q'),# scale=alt.Scale(type='log')),
    color='Country:N',
    tooltip = list(_data),
    opacity=alt.condition(selection,alt.value(1), alt.value(0.2))
).add_selection(selection).properties(
    width = 600
)

alt.Chart(...)

In [18]:
#hide_input
_df_objs = [
              df_deaths[df_deaths['Country'].isin(countries)].set_index(['Country', 'Date']).rename(columns={'Cases': 'Deaths'}),
              df_confirmed[df_confirmed['Country'].isin(countries)].set_index(['Country', 'Date']).rename(columns={'Cases': 'Confirmed'}) 
           ]
_df_countries = pd.concat(_df_objs, join='inner', axis=1).reset_index()
_df_countries['Deaths Rate'] = _df_countries['Deaths'] / _df_countries['Confirmed']
alt.Chart(_df_countries.fillna(0)).mark_line().encode(
    alt.X('Date:T'),
    alt.Y('Deaths Rate:Q', axis=alt.Axis(format='%')),
    color='Country',
    tooltip=list(_df_countries),
    opacity=alt.condition(selection,alt.value(1), alt.value(0.3))
).add_selection(selection).properties(
    width = 600
)

alt.Chart(...)